In [1]:
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import imblearn
import numpy as np

In [6]:
with open('drive/MyDrive/Diplom/test3.csv') as f:
    test = pd.read_csv(f,sep='\t')
with open('drive/MyDrive/Diplom/train3.csv') as f:
    train = pd.read_csv(f,sep='\t')
with open('drive/MyDrive/Diplom/results.csv') as f:
    results = pd.read_csv(f)

In [9]:
results

,Unnamed: 0,features,f1_score_macro,f1_score_micro,f1_score_weighted,balanced_accuracy_score,neg_log_loss,roc_auc_score ovr,roc_auc_score ovo
0,0.0,"ups,score,body_length",0.4324,0.6345,0.7000,0.5846,-1.0150,0.7410,0.7540
1,0.0,+TF-IDF+pol_and_sub,0.4995,0.7097,0.7644,0.6464,-0.7828,0.8496,0.8473
0,NaN,+LDA,0.5471,0.7749,0.8085,0.6665,-0.6980,0.8767,0.8729


In [10]:
results.drop(columns = ['Unnamed: 0'], inplace = True)

In [12]:
with open('drive/MyDrive/Diplom/test.csv') as f:
    test0 = pd.read_csv(f,sep='\t')
with open('drive/MyDrive/Diplom/train.csv') as f:
    train0 = pd.read_csv(f,sep='\t')

In [13]:
import re

pattern = '[^a-zA-Z0-9\t\n\r\f\v]'

def num_punct_symbols(text):
  return(len(re.findall(pattern, text)))

train['punct_symbols_frac'] = train0['body'].apply(num_punct_symbols)/train0['body_length']
test['punct_symbols_frac'] = test0['body'].apply(num_punct_symbols)/test0['body_length']

In [15]:
cols = np.array(train.drop(columns = ['Unnamed: 0', 'Y']).columns)

In [16]:
X_train = train[cols]
y_train = train.Y
X_test = test[cols]
y_test = test.Y

In [17]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, log_loss, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def metrics(y_test, y_pred, y_pred_proba, name):
  
  #conf = confusion_matrix(y_test, y_pred)
  #print(conf)
  f1_ma = round(f1_score(y_test, y_pred, average='macro'), 4)
  f1_mi = round(f1_score(y_test, y_pred, average='micro'), 4)
  f1_we = round(f1_score(y_test, y_pred, average='weighted'), 4)
  b_acc = round(balanced_accuracy_score(y_test, y_pred), 4)
  neg_ll = round(- log_loss(y_test, y_pred_proba), 4)
  roc_auc_ovr = round(roc_auc_score(y_test, y_pred_proba, average='weighted', multi_class = 'ovr'), 4)
  roc_auc_ovo = round(roc_auc_score(y_test, y_pred_proba, average='weighted', multi_class = 'ovo'), 4)

  return pd.DataFrame({'features': [name],'f1_score_macro': [f1_ma], 'f1_score_micro': [f1_mi],'f1_score_weighted': [f1_we], 'balanced_accuracy_score': [b_acc], 'neg_log_loss': [neg_ll],'roc_auc_score ovr': [roc_auc_ovr], 'roc_auc_score ovo': [roc_auc_ovo]})


In [18]:
pipel = Pipeline(steps = [('scaler', StandardScaler()), ('model',LogisticRegression(max_iter = 10000, class_weight = 'balanced'))])

pipel.fit(X_train, y_train)
y_pred = pipel.predict(X_test)
y_pred_proba = pipel.predict_proba(X_test)
res = metrics(y_test, y_pred, y_pred_proba, '+punct_symbols_frac')
results = pd.concat([results, res])

In [20]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks, EditedNearestNeighbours

for sampler in [RandomOverSampler(), SMOTE(), ADASYN()]:
  X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
  pipel.fit(X_resampled, y_resampled)
  y_pred = pipel.predict(X_test)
  y_pred_proba = pipel.predict_proba(X_test)
  res = metrics(y_test, y_pred, y_pred_proba, sampler.__class__.__name__)
  results = pd.concat([results, res])

In [21]:
for sampler in [RandomUnderSampler(), NearMiss(version=1),NearMiss(version=2),TomekLinks(),EditedNearestNeighbours()]:
  X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
  pipel.fit(X_resampled, y_resampled)
  y_pred = pipel.predict(X_test)
  y_pred_proba = pipel.predict_proba(X_test)
  res = metrics(y_test, y_pred, y_pred_proba, sampler.__class__.__name__)
  results = pd.concat([results, res])

In [22]:
results

,features,f1_score_macro,f1_score_micro,f1_score_weighted,balanced_accuracy_score,neg_log_loss,roc_auc_score ovr,roc_auc_score ovo
0,"ups,score,body_length",0.4324,0.6345,0.7000,0.5846,-1.0150,0.7410,0.7540
1,+TF-IDF+pol_and_sub,0.4995,0.7097,0.7644,0.6464,-0.7828,0.8496,0.8473
0,+LDA,0.5471,0.7749,0.8085,0.6665,-0.6980,0.8767,0.8729
0,+punct_symbols_frac,0.5580,0.7831,0.8131,0.6772,-0.6811,0.8792,0.8771
0,RandomOverSampler,0.5496,0.7792,0.8083,0.6633,-0.6898,0.8766,0.8734
0,SMOTE,0.5564,0.7847,0.8117,0.6621,-0.6813,0.8804,0.8755
0,ADASYN,0.5486,0.7780,0.8045,0.6474,-0.6765,0.8785,0.8727
0,RandomUnderSampler,0.5314,0.7682,0.7971,0.6363,-0.7175,0.8700,0.8628
0,NearMiss,0.2570,0.2703,0.3820,0.3264,-4.8589,0.4747,0.4829
0,NearMiss,0.4844,0.6988,0.7459,0.6185,-0.8483,0.8358,0.8356


In [23]:
results.to_csv('drive/MyDrive/Diplom/results.csv')